In [ ]:
#pip install psycopg2
'''dbname='postgres'
user='postgres'
password='root'
host = 'localhost'
port = '5432'
'''

In [ ]:
import psycopg2

dbname='postgres'
user='postgres'
password='root'
host = 'localhost'
port = '5432'
# Criando a Coneção com o nosso banco de dados
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password= password,
                        host = host,
                        port = port)

# Fazendo a coneção 
cur = conn.cursor()


#treco do codigo que vamos passar para nosso banco de dados 

#Validaando os recusros que passamos para nosso banco de dados 
conn.commit()


# fechando a coneção pera que não fique rodando sem precisar
cur.close()
conn.close()



In [2]:
#Simulando erro obs: se tiver encoding utf-8 padrao o pandas ler sem prescisar passar parametro encoding
import pandas as pd

#pd.set_option('display.max_columns', None)# ver todas as colunas  
caminho_do_arquivo = r"C:\Users\TIAGO\Documents\curso_py&pg\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig') 
df.head()

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrência,Data_da_Ocorrencia,Hora_da_Ocorrência,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,5849,0853/2014,MARCIO AGUIAR DA SILVA,Incidente Grave,2014-05-13,16:50:00,SINOP,MT,Centro-Oeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,...,0.0,0.0,0.0,210N,L1P,C210,1724.0,6.0,CESSNA AIRCRAFT,verdadeiro
1,5849,0853/2014,MARCIO AGUIAR DA SILVA,Incidente Grave,2014-05-13,16:50:00,SINOP,MT,Centro-Oeste,EXCURSÃO DE PISTA,...,0.0,0.0,0.0,210N,L1P,C210,1724.0,6.0,CESSNA AIRCRAFT,verdadeiro
2,5845,0855/2014,AERO AGRICOLA SANTOS DUMONT LTDA,Acidente,2014-05-10,20:25:00,CACHOEIRA DO SUL,RS,Sul,PERDA DE CONTROLE EM VOO,...,0.0,0.0,0.0,140A,L1P,C140,680.0,2.0,CESSNA AIRCRAFT,verdadeiro
3,5843,0810/2014,AVIACAO AGRICOLA SANTA TEREZINHA,Incidente Grave,2014-05-09,16:05:00,CARAZINHO,RS,Sul,EXCURSÃO DE PISTA,...,0.0,0.0,0.0,EMB-201A,L1P,IPAN,1800.0,1.0,NEIVA,verdadeiro
4,5390,/2003,None,Acidente,2003-12-09,None,None,Indeterminado,None,None,...,0.0,0.0,0.0,None,None,None,NaN,NaN,None,verdadeiro
